# WZGW Projekt 5 - Transformacje współrzędnych między elipsoidami. 

## flH(GRS80) -> XYZ -Transformacja-> XYZ(Krasowski) -> flH 
Nr indeksu: 311512

__Cel ćwiczenia:__ Przetransformować podane współrzędne z elipsoidy GRS80 na elipsoidę Krasowskiego. Zaprezentować wyniki.

__Transformacj aprzez podobieństwo Bursy-Wolfa:__ pozwala na transformacje współrzędnych między elipsoidami. Jest to liniowa transformacja konforemna - realizuje sztywne ruchy: obrót, przesunięcie. Ponadto przeskalowuje współrzędne (w każdym kierunku/osi jednakowo). 

__Algorytm Hirvonena:__ przyjmuje współrzędne x, y, z (oraz parametry elipsoidy) jako dane. Zwraca współrzędne geodezyjne fi, lambda, h na danej elipsoidzie. 

In [116]:
import numpy as np
from math import *
import pandas as pd

In [117]:
def geo2xyz(fi, lam, h, a=6378137, e2=0.00669437999013):
    """funkcja zamienia współrzędne geodezyjne na kartezjańskie fi, lam podajemy w radianach do wzorów, do fcji w deg"""
    # Krasowski - a=6378245, e2=0.0066934215520398155
    # GRS80 - a=6378137, e2=0.00669437999013
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    x = (N + h) * np.cos(fi) * np.cos(lam)
    y = (N + h) * np.cos(fi) * np.sin(lam)
    z = (N * (1 - e2) + h) * np.sin(fi)
    return np.array([x, y, z])

In [118]:
def bursa_wolf(xp, yp, zp, kappa=0.8407728*10**(-6), alfa=-0.35867, beta=-0.05283, gamma=0.84354, x0=-33.4297, y0=146.5746, z0=76.2865):
    """Transformacja przez podobieństwo Bursy-Wolfa.
    Realizuje sztywne ruchy (obrót i przesunięcie).
    Przeskalowanie wsp. układu pierwotnego (jedna skalaa dla wszystkich osi).
    Wiernokątna transformacja.
    xp, yp, zp - wsp punktu do transformacji.
    kappa, alfa, beta, gamma, x0, y0, z0 - stałe wartości dla danej elipsoidy.
    alfa, beta, gamma (kąty Eulera) podajemy w sekundach kątowych. x0, y0, z0 w metrach."""
    alfa = np.deg2rad(alfa/3600)
    beta = np.deg2rad(beta/3600)
    gamma = np.deg2rad(gamma/3600)
    return np.array([[xp], [yp], [zp]]) + np.array([[kappa, gamma, -beta],
                                    [-gamma, kappa, alfa],
                                    [beta, -alfa, kappa]]) @ np.array([[xp],[yp],[zp]]) + np.array([[x0],[y0],[z0]])

In [119]:
def Hirvonen(x, y, z, a=6378245, e2=0.0066934215520398155):
    """zwraca współrzędne fi, lambda, h. Algorytm Hirvonena"""
    #     epsilon = 0.00005/3600  # epsilon w stopniach dziesiętnych
    # Krasowski - a=6378245, e2=0.0066934215520398155
    # GRS80 - a=6378137, e2=0.00669437999013
    r = sqrt(x**2 + y**2)
    phi_old = atan((z / r) * (1 - e2)**(-1))
    while True:
        N = a / (sqrt(1 - e2 * (sin(phi_old))**2))
        h = (r / cos(phi_old)) - N
        phi_new = atan((z / r) * (1 - e2 * (N / (N + h)))**(-1) )
        if abs(phi_new - phi_old) < radians(0.00005 / 3600):
            break
        else:
            phi_old = phi_new
    lam = np.arctan2(y, x)
    N = a / (sqrt(1 - e2 * (sin(phi_new))**2))
    h = (r / cos(phi_new)) - N
    return degrees(phi_new), degrees(lam), h

In [120]:
def pprintdms(wsp):
    """printuje/returnuje współrzędną w formacie stopnie, minuty, sekundy kątowe. Wejście to stopnie dziesiętne"""
    d = floor(wsp)
    m = floor((wsp-d)*60)
    s = round((wsp-d-m/60)*3600, 5)
#     print(f"{d}°{m}′{s}″")          # 
    s = '{:.5f}'.format(s)       #
    return f"{d}°{m}′{s}″"

In [121]:
def dms_to_deg(d,m,s):
    """zwraca stopnie dziesiętne z dms"""
    return d+m/60+s/3600

In [122]:
# Krasowski - a=6378245, e2=0.0066934215520398155
# GRS80 - a=6378137, e2=0.00669437999013

## Testy funkcji

In [123]:
# geo2xyz(52.25, 20.75, 100)

In [124]:
# bursa_wolf(*geo2xyz(52.25, 20.75, 100))

In [125]:
# Hirvonen(*bursa_wolf(*geo2xyz(52.25, 20.75, 100)), a=6378245, e2=0.0066934215520398155)

In [126]:
# Hirvonen(3821515.256841614, 1447842.614679356, 4880703.023862205, a=6378245, e2=0.0066934215520398155)

In [127]:
# Hirvonen(3821515.256841614, 1447842.614679356, 4880703.023862205, a=6378245, e2=0.0066934215520398155)

In [128]:
# Hirvonen(*geo2xyz(50.25000000000001, 20.75, 0, a=6378245, e2=0.0066934215520398155), a=6378245, e2=0.0066934215520398155)

## Tabelka

In [129]:
# Współrzędne GRS80 do transformacji na Krasowskiego (takie same są w .csv)
A = [50.25, 20.75]
B = [50, 20.75]
C = [50.25, 21.25]
D = [50, 21.25]
sr_szer = [50.125, 21]
srodkowy = [50.1252704344052, 21.0006510711438]

In [130]:
df = pd.read_csv('dane.csv', sep=';')
df

,fi_g,lambda_g,h_g
0,50.25000,20.750000,100
1,50.00000,20.750000,100
2,50.25000,21.250000,100
3,50.00000,21.250000,100
4,50.12500,21.000000,100
5,50.12527,21.000651,100


In [131]:
df['xyz_g'] = df.apply(lambda row: geo2xyz(row[0], row[1], row[2]), axis=1)
df

,fi_g,lambda_g,h_g,xyz_g
0,50.25000,20.750000,100,"[3821511.431829001, 1447841.1655117115, 488069..."
1,50.00000,20.750000,100,"[3841468.4576248974, 1455402.2062161348, 48628..."
2,50.25000,21.250000,100,"[3808731.283056597, 1481134.5913238598, 488069..."
3,50.00000,21.250000,100,"[3828621.5672599915, 1488869.499821071, 486286..."
4,50.12500,21.000000,100,"[3825128.783106747, 1468329.3692134656, 487179..."
5,50.12527,21.000651,100,"[3825090.545402917, 1468364.5618904294, 487181..."


In [132]:
df['xyz_k'] = df.apply(lambda row: bursa_wolf(*row[3]), axis=1)
df

,fi_g,lambda_g,h_g,xyz_g,xyz_k
0,50.25000,20.750000,100,"[3821511.431829001, 1447841.1655117115, 488069...","[[3821488.3863170543], [1447964.8420271617], [..."
1,50.00000,20.750000,100,"[3841468.4576248974, 1455402.2062161348, 48628...","[[3841445.4552475763], [1455525.8384738027], [..."
2,50.25000,21.250000,100,"[3808731.283056597, 1481134.5913238598, 488069...","[[3808708.362956154], [1481258.348097179], [48..."
3,50.00000,21.250000,100,"[3828621.5672599915, 1488869.499821071, 486286...","[[3828598.690949109], [1488993.2127557376], [4..."
4,50.12500,21.000000,100,"[3825128.783106747, 1468329.3692134656, 487179...","[[3825105.8221444753], [1468453.0636419074], [..."
5,50.12527,21.000651,100,"[3825090.545402917, 1468364.5618904294, 487181...","[[3825067.5845573596], [1468488.2564713017], [..."


In [133]:
df['fi_k'] = df.apply(lambda row: Hirvonen(*row[4])[0] , axis=1)
df['lambda_k'] = df.apply(lambda row: Hirvonen(*row[4])[1] , axis=1)
df['h_k'] = df.apply(lambda row: Hirvonen(*row[4])[2] , axis=1)
df

,fi_g,lambda_g,h_g,xyz_g,xyz_k,fi_k,lambda_k,h_k
0,50.25000,20.750000,100,"[3821511.431829001, 1447841.1655117115, 488069...","[[3821488.3863170543], [1447964.8420271617], [...",50.250293,20.751736,67.633426
1,50.00000,20.750000,100,"[3841468.4576248974, 1455402.2062161348, 48628...","[[3841445.4552475763], [1455525.8384738027], [...",50.000296,20.751726,67.500946
2,50.25000,21.250000,100,"[3808731.283056597, 1481134.5913238598, 488069...","[[3808708.362956154], [1481258.348097179], [48...",50.250285,21.251734,68.464166
3,50.00000,21.250000,100,"[3828621.5672599915, 1488869.499821071, 486286...","[[3828598.690949109], [1488993.2127557376], [4...",50.000288,21.251724,68.336035
4,50.12500,21.000000,100,"[3825128.783106747, 1468329.3692134656, 487179...","[[3825105.8221444753], [1468453.0636419074], [...",50.125290,21.001730,67.983943
5,50.12527,21.000651,100,"[3825090.545402917, 1468364.5618904294, 487181...","[[3825067.5845573596], [1468488.2564713017], [...",50.125561,21.002381,67.985169


In [134]:
pprint_df = df.drop(columns=['xyz_g', 'xyz_k'])
pprint_df['fi_grs80'] = pprint_df.apply(lambda row: pprintdms(row[0]) , axis=1)
pprint_df['lam_grs80'] = pprint_df.apply(lambda row: pprintdms(row[1]) , axis=1)
pprint_df['h_grs80'] = pprint_df.apply(lambda row: round(row[2],3) , axis=1)
pprint_df['fi_kraso'] = pprint_df.apply(lambda row: pprintdms(row[3]) , axis=1)
pprint_df['lam_kraso'] = pprint_df.apply(lambda row: pprintdms(row[4]) , axis=1)
pprint_df['h_kraso'] = pprint_df.apply(lambda row: round(row[-6],3) , axis=1)
pprint_df.drop(columns=['fi_g','lambda_g','h_g','fi_k','lambda_k','h_k'], inplace=True)
pprint_df

,fi_grs80,lam_grs80,h_grs80,fi_kraso,lam_kraso,h_kraso
0,50°15′0.00000″,20°45′0.00000″,100,50°15′1.05525″,20°45′6.24956″,67.633
1,50°0′0.00000″,20°45′0.00000″,100,50°0′1.06530″,20°45′6.21425″,67.501
2,50°15′0.00000″,21°15′0.00000″,100,50°15′1.02602″,21°15′6.24100″,68.464
3,50°0′0.00000″,21°15′0.00000″,100,50°0′1.03618″,21°15′6.20572″,68.336
4,50°7′30.00000″,21°0′0.00000″,100,50°7′31.04568″,21°0′6.22764″,67.984
5,50°7′30.97356″,21°0′2.34386″,100,50°7′32.01920″,21°0′8.57152″,67.985


__Wnioski:__ Widać, że współrzędne na elipsoidzie GRS80 po transformacji na elipsoidę Krasowskiego różnią się. Szczególnie to widać na wysokościach. Wysokośći na elipsoidzie Krasowskiego są mniejsze niż na GRS80, co pozwala stwierdzić, że elipsoida Krasowskiego w pewnym obszarze Polski (nie testowaliśmy punktów z różnych krańców Polski, ale pewnie jest tak dla całej Polski) jest "wyżej" niż GRS80. Poza tym, poznałem transformację Bursy-Wolfa, która wykorzystując działania na macierzach, transformuje współrzędne XYZ z jednej elipsoidy na drugą.  